# ParFuMor (version YAML & Objets)

- Ajout d'un %store pour la gestion des numéros de kanoniks (16/04/20)

- Attention les règles qui ne changent pas le radical donnent lieu à des mauvais découpages des formes...

In [460]:
# -*- coding: utf8 -*-
import os
from os.path import expanduser
import itertools
import yaml,YamlDuplicates
from yaml.constructor import ConstructorError
import warnings
import ParFuMor as PFM
from ParFuMor import *
import pickle
from IPython.display import HTML, display
#import cellbell 

In [461]:
def ding():
    os.system('afplay /System/Library/Sounds/Submarine.aiff')

# Gestion partagée des numéros à traiter

le block suivant permet de partager les numéros à traiter entre les différents Notebooks.
- %store -r variable lit la variable dans le stock
- %store variable stocke la variable

In [462]:
%store -r numerosKalaba
numerosKalaba=[1,2,3,4,5]
numerosKalaba=[4]
print numerosKalaba
%store numerosKalaba

[4]
Stored 'numerosKalaba' (list)


In [463]:
home = expanduser("~")
repertoire=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/"
serie=repertoire+"21-"
nomsKalabas=[serie+"K%d/"%num for num in numerosKalaba]

In [464]:
def getExemple(lDict):
    exKey=lDict.keys()[0]
    if isinstance(lDict[exKey],dict):
        result=getExemple(lDict[exKey])
    elif isinstance(lDict[exKey],list):
        result=lDict[exKey][0]
    return result

In [465]:
PFM.duplicateErrors=[]
for serie in nomsKalabas:
    print
    print serie

    with open(serie+"Gloses.yaml", 'r') as stream:
        gloses=yaml.safe_load(stream)
        PFM.gloses=gloses
    with open(serie+"Stems.yaml", 'r') as stream:
        try:
            stems=yaml.safe_load(stream)
            PFM.stems=stems
        except ConstructorError,msg:
            print msg
            continue
    lexiqueTestPrep=getExemple(stems["PREP"])
    lexiqueTestNoun=getExemple(stems["NOM"])
    lexiqueTestHyper=getExemple(stems["ADJ"])
    with open(serie+"Blocks.yaml", 'r') as stream:
        blocks=yaml.safe_load(stream)
        PFM.blocks=blocks
    with open(serie+"Phonology.yaml", 'r') as stream:
        phonology=yaml.safe_load(stream)
        PFM.phonology=phonology
    with open(serie+"MorphoSyntax.yaml", 'r') as stream:
        morphosyntax=yaml.safe_load(stream)
        PFM.morphosyntax=morphosyntax

        
    for cat in morphosyntax["Attributs"]:
        if sorted(morphosyntax["Attributs"][cat])!=sorted(gloses[cat].keys()):
            warnings.warn("\n"+serie+"\nLes attributs de %s ne sont pas cohérents\nMorphosyntax => %s\nGloses => %s"%(cat,", ".join(morphosyntax["Attributs"][cat]),", ".join(gloses[cat].keys())))
    
    regles=Regles()
    PFM.regles=regles
    for categorie in blocks:
        regles.addBlocs(categorie,blocks[categorie])


    paradigmes=Paradigmes()
    PFM.paradigmes=paradigmes

    hierarchieCF=HierarchieCF()
    PFM.hierarchieCF=hierarchieCF
    lexique=Lexique()
    PFM.lexique=lexique


    for cat in gloses:
    #    print cat
        attributes=[]
        if gloses[cat]:
            if set(gloses[cat].keys())==set(morphosyntax["Attributs"][cat]):
                features=morphosyntax["Attributs"][cat]
    #            print "inhérent",features
            else:
                features=gloses[cat].keys()
    #            print "contextuel",features
            for attribute in features:
                attributes.append(gloses[cat][attribute])
            nuplets=(itertools.product(*attributes))
            for nuplet in nuplets:
                proprietes=[cat]
                for element in range(len(nuplet)):
                    proprietes.append(u"%s=%s"%(features[element],nuplet[element]))
                paradigmes.addForme(cat,proprietes)

    analyserGloses(gloses)
    analyserStems(stems)
        
    with open(serie+"Hierarchie-S2.pkl", 'wb') as output:
       pickle.dump(hierarchieCF, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Lexique-S2.pkl", 'wb') as output:
       pickle.dump(lexique, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Regles-S2.pkl", 'wb') as output:
       pickle.dump(regles, output, pickle.HIGHEST_PROTOCOL)


    PFM.lexique.lexemes[lexiqueTestPrep]

    mot=lexiqueTestNoun
    classesNom=PFM.lexique.formeLexeme[mot][0].split('.')[1:]
    [classeElement for classeElement in classesNom if classeElement in gloses["NOM"]["Genre"]]

    PFM.lexique.formeLexeme[lexiqueTestHyper][0]

if PFM.duplicateErrors:
    print
    print "=======ERREURS========"
    print
    for ligne in PFM.duplicateErrors:
        print ligne



/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/21-K4/


/Users/gilles/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:32: UserWarning: 
/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/21-K4/
Les attributs de ADJ ne sont pas cohérents
Morphosyntax => Genre, Nombre, Cas
Gloses => Genre, Nombre


NOM : Genre Nombre Cas
VER : Temps Pers Genre
ADJ : Genre Nombre
PRO : Genre Nombre Cas
DET : Genre Nombre Cas
head stems
head stems,NOM
head stems,NOM,INAN
head stems,NOM,HUM
head stems,NOM,ANIM
head stems,VER
head stems,VER,VI
head stems,VER,VT
head stems,ADJ


In [466]:
getExemple(stems["PREP"])

'par'

In [467]:
print PFM.lexique.formeLexeme[lexiqueTestHyper][0]
print PFM.lexique.formeLexeme[lexiqueTestNoun][0]
print PFM.lexique.formeLexeme[lexiqueTestPrep][0]

profond.A1
lance.INAN.N1
par


In [468]:
paradigmes.getSigmas(["VER"])

[u'VER, Temps=PRS, Pers=3Sg, Genre=HUM',
 u'VER, Temps=PRS, Pers=3Sg, Genre=ANIM',
 u'VER, Temps=PRS, Pers=3Sg, Genre=INAN',
 u'VER, Temps=PRS, Pers=3Pau, Genre=HUM',
 u'VER, Temps=PRS, Pers=3Pau, Genre=ANIM',
 u'VER, Temps=PRS, Pers=3Pau, Genre=INAN',
 u'VER, Temps=PRS, Pers=3Pl, Genre=HUM',
 u'VER, Temps=PRS, Pers=3Pl, Genre=ANIM',
 u'VER, Temps=PRS, Pers=3Pl, Genre=INAN',
 u'VER, Temps=PST, Pers=3Sg, Genre=HUM',
 u'VER, Temps=PST, Pers=3Sg, Genre=ANIM',
 u'VER, Temps=PST, Pers=3Sg, Genre=INAN',
 u'VER, Temps=PST, Pers=3Pau, Genre=HUM',
 u'VER, Temps=PST, Pers=3Pau, Genre=ANIM',
 u'VER, Temps=PST, Pers=3Pau, Genre=INAN',
 u'VER, Temps=PST, Pers=3Pl, Genre=HUM',
 u'VER, Temps=PST, Pers=3Pl, Genre=ANIM',
 u'VER, Temps=PST, Pers=3Pl, Genre=INAN']

In [469]:
case="CF=N3"
m=re.match(ur"(.*=)(.*)","CF=N1|N2")
if m:
    altTrait=m.group(1)
    altValeurs=m.group(2).split("|")
    print altTrait
    altTraits=[altTrait+v for v in altValeurs]
    print altTraits
    if any(t == case for t in altTraits):
        print "applies"
    else:
        print "does not apply"

CF=
['CF=N1', 'CF=N2']
does not apply


In [470]:
regles.getRules("VER",'CF=V1, Temps=PRS, Pers=3PL')
# regles.getRules("PRO","Nombre=Du, Genre=C")

[('X+b', 'CF=V1,Temps=PRS')]

In [471]:
test=Lexeme("ganav","VER","croc").getParadigm(lignes=["Genre"],colonne=["Pers","Temps"])
test=Lexeme("ganav","NOM","croc").getParadigm(lignes=["Nombre"],colonne=["Cas"])

# test=Lexeme("RAD","DET","croc").getParadigm(lignes=["Genre"],colonne=["Nombre","Cas"])
test
#test.index.sortlevel(0,ascending=False)

del Genre


Cas,Abs,Dat,Erg,Obl
Nombre,,,,
Pau,ganav,ganav-v,ganav-Z,ganav
Pl,ganav-we,ganav-we-v,ganav-we,ganav-we
Sg,ganav,ganav-v,ganav,ganav-d


In [472]:
cat="N"
triLex={}
for element in PFM.lexique.lexemes:
#    print element
    elementElts=element.split(".")
    nom=elementElts[0]
    if len(elementElts)==3:
        if elementElts[1].startswith(cat):
            if not elementElts[2] in triLex:
                triLex[elementElts[2]]=set()
            triLex[elementElts[2]].add((PFM.lexique.lexemes[element].stem,nom))            
triLex

{}

In [473]:
for l in PFM.lexique.lexemes:
    print l,PFM.lexique.lexemes[l].stem

arbre.INAN.N1 NuSoT
fuir.VT.V1 Zetak
courageux.A2 fugur
apporter.VT.V1 ZojuZ
enfant.HUM.N3 libor
œuf.INAN.N3 tuzeN
pendre.VT.V1 Duwif
viande.INAN.N1 Tagol
placer.VT.V2 makuS
douleur.INAN.N3 segot
chambre.INAN.N3 tamar
gros.A1 dobal
ressusciter.VT.V2 naveZ
voler.VT.V1 Zukub
retourner.VI.V1 bagut
reconstruire.VT.V2 posaf
par bej
gargouille.ANIM.N2 gevop
lueur.INAN.N1 Zuzab
lit.INAN.N1 DaZum
combattant.HUM.N1 beneg
Violette.HUM.N1 vjolet
nourrir.VT.V2 mutase
inquiéter.VT.V2 voriT
échanger.VT.V2 siduT
louve.ANIM.N2 Tuvir
rejoindre.VT.V2 riluS
trois.A2 poker
feu.ANIM.N3 kobud
croc.INAN.N3 sojal
présenter.VT.V1 bemud
changer.VT.V2 tedut
nombreux.A2 bekut
reconnaître.VT.V1 Dagam
sortir.VT.V2 nufuS
jaune.A2 kubon
blanc.A1 povov
planter.VT.V2 goTek
grand.A1 savat
courir.VI.V2 tidag
pleurer-act.VI.V1 Sosak
mort.INAN.N3 Soniv
dernier-nom.INAN.N3 ruwoN
effrayé.A2 nagor
furieux.A1 bimog
tornade.ANIM.N2 moger
disparaître.VT.V2 toZune
deux.A1 loNak
fille.HUM.N1 dobin
harmonie.INAN.N3 pozid
Clemencia.

In [474]:
ding()